In [1]:
include("./Utils.jl")

using MLIR
using MLIR: IR, API, API.mlir_c
using .Utils

import Base.push!

In [2]:
function pow(x::F, n::Integer) where {F}
    p = one(F)
    for _ in 1:n
        p *= x
    end
    p
end

pow (generic function with 1 method)

In [3]:
ir, ret = @code_ircode pow(2, 10);

In [4]:
ir

│╻╷╷╷╷ Colon3 1 ── %1  = Base.sle_int(1, _3)::Bool
││┃│    UnitRange  └───       goto #3 if not %1
│││┃     unitrange_last  2 ──       goto #4
││││    3 ──       goto #4
│││     4 ┄─ %5  = φ (#2 => _3, #3 => 0)::Int64
│││     └───       goto #5
││      5 ──       goto #6
││╻╷╷   isempty  6 ── %8  = Base.slt_int(%5, 1)::Bool
││      └───       goto #8 if not %8
││      7 ──       goto #9
││      8 ──       goto #9
│       9 ┄─ %12 = φ (#7 => true, #8 => false)::Bool
│       │    %13 = φ (#8 => 1)::Int64
│       │    %14 = Base.not_int(%12)::Bool
│       └───       goto #15 if not %14
│       10 ┄ %16 = φ (#9 => %13, #14 => %24)::Int64
│       │    %17 = φ (#9 => 1, #14 => %18)::Int64
│╻     *4 │    %18 = Base.mul_int(%17, _2)::Int64
│╻╷    iterate5 │    %19 = (%16 === %5)::Bool
││      └───       goto #12 if not %19
││      11 ─       goto #13
││╻     +  12 ─ %22 = Base.add_int(%16, 1)::Int64
│╻     iterate  └───       goto #13
│       13 ┄ %24 = φ (#12 => %22)::Int64
│       │    %25 =

In [5]:
fieldnames(typeof(ir.stmts[10].data))

(:inst, :type, :info, :line, :flag)

In [6]:
ir.stmts.inst

30-element Vector{Any}:
 :(Base.sle_int(1, _3))
 :(goto %3 if not %1)
 :(goto %4)
 :(goto %4)
 :(φ (%2 => _3, %3 => 0))
 :(goto %5)
 :(goto %6)
 :(Base.slt_int(%5, 1))
 :(goto %8 if not %8)
 :(goto %9)
 ⋮
 :(Base.add_int(%16, 1))
 :(goto %13)
 :(φ (%12 => %22))
 :(φ (%11 => true, %12 => false))
 :(Base.not_int(%25))
 :(goto %15 if not %26)
 :(goto %10)
 :(φ (%13 => %18, %9 => 1))
 :(return %29)

In [7]:
ir.cfg.blocks

15-element Vector{Core.Compiler.BasicBlock}:
 Core.Compiler.BasicBlock([1, 2], Int64[], [3, 2])
 Core.Compiler.BasicBlock([3], [1], [4])
 Core.Compiler.BasicBlock([4], [1], [4])
 Core.Compiler.BasicBlock([5, 6], [2, 3], [5])
 Core.Compiler.BasicBlock([7], [4], [6])
 Core.Compiler.BasicBlock([8, 9], [5], [8, 7])
 Core.Compiler.BasicBlock([10], [6], [9])
 Core.Compiler.BasicBlock([11], [6], [9])
 Core.Compiler.BasicBlock([12, 13, 14, 15], [7, 8], [15, 10])
 Core.Compiler.BasicBlock([16, 17, 18, 19, 20], [9, 14], [12, 11])
 Core.Compiler.BasicBlock([21], [10], [13])
 Core.Compiler.BasicBlock([22, 23], [10], [13])
 Core.Compiler.BasicBlock([24, 25, 26, 27], [11, 12], [15, 14])
 Core.Compiler.BasicBlock([28], [13], [10])
 Core.Compiler.BasicBlock([29, 30], [9, 13], Int64[])

In [8]:
ir.cfg.blocks[1] |> (fieldnames ∘ typeof)

(:stmts, :preds, :succs)

In [9]:
f(x, y) = 2*(x+y)

f (generic function with 1 method)

In [10]:
ir, ret = @code_ircode f(2, 3)

│╻ +1 1 ─ %1 = Base.add_int(_2, _3)::Int64
│╻ *  │   %2 = Base.mul_int(2, %1)::Int64
│   └──      return %2
     => Int64

In [11]:
function registerAllDialects!(ctx)
    registry = MLIR.API.mlirDialectRegistryCreate()
    MLIR.API.mlirRegisterAllDialects(registry)
    handle = MLIR.API.mlirGetDialectHandle__jlir__()
    API.mlirDialectHandleInsertDialect(handle, registry)
    # MLIR.API.mlirDialectHandleRegisterDialect(handle, ctx)
    MLIR.API.mlirContextAppendDialectRegistry(ctx, registry)
    # MLIR.API.mlirDialectRegistryDestroy(registry)

    MLIR.API.mlirContextLoadAllAvailableDialects(ctx)
    return registry
end

registerAllDialects! (generic function with 1 method)

In [12]:
# TODO: the difference between loaddialect and registerdialect is unclear to me.

ctx = IR.Context()
registry = registerAllDialects!(ctx)

# let 
#     handle = @ccall mlir_c.mlirGetDialectHandle__jlir__()::IR.MlirDialectHandle
#     API.mlirDialectHandleLoadDialect(handle, ctx)

#     handle = @ccall mlir_c.mlirGetDialectHandle__func__()::IR.MlirDialectHandle
#     API.mlirDialectHandleLoadDialect(handle, ctx)
# end

MLIR.API.MlirDialectRegistry(Ptr{Nothing} @0x000055d828120cd0)

In [13]:
# TODO: provide sensible location

state = Ref(API.mlirOperationStateGet("func.func", IR.Location(ctx)))

Base.RefValue{MLIR.API.MlirOperationState}(MLIR.API.MlirOperationState(MLIR.API.MlirStringRef(Cstring(0x00007f8edc050360), 0x0000000000000009), MLIR.API.MlirLocation(Ptr{Nothing} @0x000055d827df6890), 0, Ptr{MLIR.API.MlirType} @0x0000000000000000, 0, Ptr{MLIR.API.MlirValue} @0x0000000000000000, 0, Ptr{MLIR.API.MlirRegion} @0x0000000000000000, 0, Ptr{MLIR.API.MlirBlock} @0x0000000000000000, 0, Ptr{MLIR.API.MlirNamedAttribute} @0x0000000000000000, false))

In [14]:
function API.MlirType(ctx::MLIR.IR.Context, t)
    return @ccall mlir_c.brutus_get_jlirtype(ctx::MLIR.IR.Context, t::Any)::API.MlirType
end

In [15]:
args = let
    argtypes = getfield(ir, :argtypes)
    API.MlirType.(Ref(ctx), argtypes)
end

3-element Vector{MLIR.API.MlirType}:
 MLIR.API.MlirType(Ptr{Nothing} @0x000055d82875abc0)
 MLIR.API.MlirType(Ptr{Nothing} @0x000055d828101380)
 MLIR.API.MlirType(Ptr{Nothing} @0x000055d828101380)

In [16]:
# TODO: functions with multiple blocks need more blocks.

reg = API.mlirRegionCreate()
entry_block = API.mlirBlockCreate(length(args), args, [IR.Location(ctx) for _ in enumerate(args)])

API.mlirRegionAppendOwnedBlock(reg, entry_block)
API.mlirOperationStateAddOwnedRegions(state, 1, [reg])

In [17]:
push!(block::API.MlirBlock, type::API.MlirType, loc::API.MlirLocation) =
    API.mlirBlockAddArgument(block, type, loc)

push! (generic function with 39 methods)

In [18]:
# args = push!.(Ref(entry_block), API.MlirType.(Ref(ctx), ir.argtypes), Ref(IR.Location(ctx).location))
input_types = API.mlirBlockGetArgument.(Ref(entry_block), eachindex(ir.argtypes) .- 1)

3-element Vector{MLIR.API.MlirValue}:
 MLIR.API.MlirValue(Ptr{Nothing} @0x000055d8273cdd40)
 MLIR.API.MlirValue(Ptr{Nothing} @0x000055d827e8e4e0)
 MLIR.API.MlirValue(Ptr{Nothing} @0x000055d82862a000)

In [19]:
API.mlirBlockGetNumArguments(entry_block)

3

In [20]:
push!(block::API.MlirBlock, op::API.MlirOperation) = 
    API.mlirBlockAppendOwnedOperation(block, op)

push! (generic function with 40 methods)

In [21]:
ret_op = IR.create_operation("func.return", IR.Location(ctx); operands=[args[2]], result_inference = false).operation
push!(entry_block, ret_op)

In [22]:
# TODO: is `ret` always a single value?
named_type_attr = let 
    function_type = API.mlirFunctionTypeGet(
        ctx,
        length(input_types), API.mlirValueGetType.(input_types),
        1, [API.MlirType(ctx, ret)])
        
    type_attr = API.mlirTypeAttrGet(function_type)
    
    API.mlirNamedAttributeGet(API.mlirIdentifierGet(ctx, "function_type"), type_attr)
end

MLIR.API.MlirNamedAttribute(MLIR.API.MlirIdentifier(Ptr{Nothing} @0x000055d8274e7058), MLIR.API.MlirAttribute(Ptr{Nothing} @0x000055d826b694e0))

In [23]:
named_symbol_name_attr = let 
    name = "f"
    
    symbol_name_attr = API.mlirStringAttrGet(ctx, name)
    
    API.mlirNamedAttributeGet(API.mlirIdentifierGet(ctx, "sym_name"), symbol_name_attr)
end

MLIR.API.MlirNamedAttribute(MLIR.API.MlirIdentifier(Ptr{Nothing} @0x000055d8274e6740), MLIR.API.MlirAttribute(Ptr{Nothing} @0x000055d8289df180))

In [24]:
named_viz_attr = let
    viz_attr = API.mlirStringAttrGet(ctx, "nested")
    
    API.mlirNamedAttributeGet(API.mlirIdentifierGet(ctx, "sym_visibility"), viz_attr)
end

MLIR.API.MlirNamedAttribute(MLIR.API.MlirIdentifier(Ptr{Nothing} @0x000055d827117540), MLIR.API.MlirAttribute(Ptr{Nothing} @0x000055d828990c40))

In [25]:
named_unit_attr = let 
    unit_attr = API.mlirUnitAttrGet(ctx)

    API.mlirNamedAttributeGet(API.mlirIdentifierGet(ctx, "llvm.emit_c_interface"), unit_attr)
end

MLIR.API.MlirNamedAttribute(MLIR.API.MlirIdentifier(Ptr{Nothing} @0x000055d828990c70), MLIR.API.MlirAttribute(Ptr{Nothing} @0x000055d827df6880))

In [26]:
function push!(state::Base.RefValue{MLIR.API.MlirOperationState}, attr::IR.MlirNamedAttribute)
    API.mlirOperationStateAddAttributes(state, 1, Ref(attr))
end

push! (generic function with 41 methods)

In [27]:
push!(state, named_type_attr)
push!(state, named_symbol_name_attr)
push!(state, named_viz_attr)
push!(state, named_unit_attr)

In [29]:
op = API.mlirOperationCreate(state)

In [55]:
dump(state)

Base.RefValue{MLIR.API.MlirOperationState}
  x: MLIR.API.MlirOperationState
    name: MLIR.API.MlirStringRef
      data: Cstring Cstring(0x00007f224bc52580)
      length: UInt64 0x0000000000000009
    location: MLIR.API.MlirLocation
      ptr: Ptr{Nothing} @0x000055eb4bc57360
    nResults: Int64 0
    results: Ptr{MLIR.API.MlirType} @0x0000000000000000
    nOperands: Int64 0
    operands: Ptr{MLIR.API.MlirValue} @0x0000000000000000
    nRegions: Int64 1
    regions: Ptr

{MLIR.API.MlirRegion} @0x000055eb4ac0ca50
    nSuccessors: Int64 0
    successors: Ptr{MLIR.API.MlirBlock} @0x0000000000000000
    nAttributes: Int64 4
    attributes: Ptr{MLIR.API.MlirNamedAttribute} @0x000055eb4a1be190
    enableResultTypeInference: Bool false


In [56]:
API.mlirOperationVerify(op)

error: 'func.func' op entry block must have 3 arguments to match function signature


false

In [52]:
IR.create_operation("func.return", IR.Location(ctx); operands, result_inference = false)

MLIR.API.MlirModule(Ptr{Nothing} @0x000055d47e975d50)

In [60]:
ctx = IR.Context()
 
handle = @ccall mlir_c.mlirGetDialectHandle__jlir__()::IR.MlirDialectHandle
API.mlirDialectHandleLoadDialect(handle, ctx)

API.mlirContextGetNumLoadedDialects(ctx)

2

In [181]:
registry

MLIR.API.MlirDialectRegistry(Ptr{Nothing} @0x000055d4829e11f0)

In [185]:
API.mlirContextGetNumRegisteredDialects(ctx)

38

In [51]:
ir.stmts[]

MethodError: MethodError: no method matching getindex(::Core.Compiler.InstructionStream, ::Core.Compiler.StmtRange)

Closest candidates are:
  getindex(::Core.Compiler.InstructionStream, !Matched::Int64)
   @ Base show.jl:2778


In [54]:
ir.stmts[first(ir.cfg.blocks).stmts[1]]

Core.Compiler.Instruction(Core.Compiler.InstructionStream(Any[:(Base.add_int(_2, _3)), :(Base.mul_int(2, %1)), :(return %2)], Any[Int64, Int64, Any], Core.Compiler.CallInfo[Core.Compiler.NoCallInfo(), Core.Compiler.NoCallInfo(), Core.Compiler.NoCallInfo()], Int32[2, 3, 1], UInt8[0x70, 0x70, 0x60]), 1)